## prep_lesson

In [1]:
import pandas as pd
import numpy as np
import unicodedata
import re
import json

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

import acquire

/Users/rachelreuter/ds-methodologies/nlp/acquire.py:19: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 19 of the file /Users/rachelreuter/ds-methodologies/nlp/acquire.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(response.text)


### Import our data

In [2]:
import pandas as pd
import os
import re
import requests
from requests import get
from bs4 import BeautifulSoup
from pathlib import Path

links = ['https://codeup.com/competitor-bootcamps-are-closing-is-the-model-in-danger/',
         'https://codeup.com/why-san-antonio-has-more-than-tacos-to-offer/',
         'https://codeup.com/codeups-data-science-career-accelerator-is-here/',
         'https://codeup.com/data-science-myths/',
         'https://codeup.com/data-science-vs-data-analytics-whats-the-difference/'
        ]

def get_blog_articles(url):
    headers = {'User-Agent': 'Codeup Ada Data Science'}
    response = get(url, headers=headers)
    soup = BeautifulSoup(response.text)
    title = soup.find('title')
    content = soup.find('div', class_='mk-single-content')

    d = dict()
    d['title'] = title.text
    d['content'] = content.text
    return d

corpus = []

for link in links:
    corpus.append((get_blog_articles(link)))

In [3]:
original = corpus[2]['content']

Above creates an "original" copy that we'll pull in later, down below.  Save it here for now.

Below, we create a working "article" copy of the original, that we'll perform all the transformations to.

In [7]:
article = original

In [8]:
print(article)


The rumors are true! The time has arrived. Codeup has officially opened applications to our new Data Science career accelerator, with only 25 seats available! This immersive program is one of a kind in San Antonio, and will help you land a job in Glassdoor’s #1 Best Job in America.
Data Science is a method of providing actionable intelligence from data. The data revolution has hit San Antonio, resulting in an explosion in Data Scientist positions across companies like USAA, Accenture, Booz Allen Hamilton, and HEB. We’ve even seen UTSA invest $70 M for a Cybersecurity Center and School of Data Science. We built a program to specifically meet the growing demands of this industry.
Our program will be 18 weeks long, full-time, hands-on, and project-based. Our curriculum development and instruction is led by Senior Data Scientist, Maggie Giust, who has worked at HEB, Capital Group, and Rackspace, along with input from dozens of practitioners and hiring partners. Students will work with rea

### Convert all to lower case

In [10]:
article = article.lower()

In [11]:
print(re.sub(r'\s', ' ',article))

 the rumors are true! the time has arrived. codeup has officially opened applications to our new data science career accelerator, with only 25 seats available! this immersive program is one of a kind in san antonio, and will help you land a job in glassdoor’s #1 best job in america. data science is a method of providing actionable intelligence from data. the data revolution has hit san antonio, resulting in an explosion in data scientist positions across companies like usaa, accenture, booz allen hamilton, and heb. we’ve even seen utsa invest $70 m for a cybersecurity center and school of data science. we built a program to specifically meet the growing demands of this industry. our program will be 18 weeks long, full-time, hands-on, and project-based. our curriculum development and instruction is led by senior data scientist, maggie giust, who has worked at heb, capital group, and rackspace, along with input from dozens of practitioners and hiring partners. students will work with rea

### Removing accented characters
In 3 steps:

NORMALIZING (unicodedata.normalize removes any inconsistenceies in unicode character encoding)

ENCODING, and (.encode to convert resulting string into the ascii character set.  Use "ignore"  for any conversion errors, meaning we'll drop anything that's not an ascii character.)

DECODING (.decode to turn resulting Python Bytes object back into a string)

### Normalizing, encoding, decoding

In [12]:
import unicodedata

In [13]:
normalized = unicodedata.normalize('NFKD', article).encode('ascii','ignore').decode('utf-8','ignore')

In [14]:
print(normalized)


the rumors are true! the time has arrived. codeup has officially opened applications to our new data science career accelerator, with only 25 seats available! this immersive program is one of a kind in san antonio, and will help you land a job in glassdoors #1 best job in america.
data science is a method of providing actionable intelligence from data. the data revolution has hit san antonio, resulting in an explosion in data scientist positions across companies like usaa, accenture, booz allen hamilton, and heb. weve even seen utsa invest $70 m for a cybersecurity center and school of data science. we built a program to specifically meet the growing demands of this industry.
our program will be 18 weeks long, full-time, hands-on, and project-based. our curriculum development and instruction is led by senior data scientist, maggie giust, who has worked at heb, capital group, and rackspace, along with input from dozens of practitioners and hiring partners. students will work with real 

### Now store it back into the original "article" name

In [15]:
article = normalized

In [16]:
article

'\nthe rumors are true! the time has arrived. codeup has officially opened applications to our new data science career accelerator, with only 25 seats available! this immersive program is one of a kind in san antonio, and will help you land a job in glassdoors #1 best job in america.\ndata science is a method of providing actionable intelligence from data. the data revolution has hit san antonio, resulting in an explosion in data scientist positions across companies like usaa, accenture, booz allen hamilton, and heb. weve even seen utsa invest $70 m for a cybersecurity center and school of data science. we built a program to specifically meet the growing demands of this industry.\nour program will be 18 weeks long, full-time, hands-on, and project-based. our curriculum development and instruction is led by senior data scientist, maggie giust, who has worked at heb, capital group, and rackspace, along with input from dozens of practitioners and hiring partners. students will work with r

### Removing Special Characters

In [17]:
article = re.sub(r"[^a-z0-9\s]", '', article)

In [18]:
print(article)


the rumors are true the time has arrived codeup has officially opened applications to our new data science career accelerator with only 25 seats available this immersive program is one of a kind in san antonio and will help you land a job in glassdoors 1 best job in america
data science is a method of providing actionable intelligence from data the data revolution has hit san antonio resulting in an explosion in data scientist positions across companies like usaa accenture booz allen hamilton and heb weve even seen utsa invest 70 m for a cybersecurity center and school of data science we built a program to specifically meet the growing demands of this industry
our program will be 18 weeks long fulltime handson and projectbased our curriculum development and instruction is led by senior data scientist maggie giust who has worked at heb capital group and rackspace along with input from dozens of practitioners and hiring partners students will work with real data sets realistic problems 

### Tokenization
The process we've just gone through, that is, removing non-ascii characters and special characters can be thought of as a kind of tokenization.

Tokenization is the process of breaking something down into discrete units. In the context of NLP, this means breaking text down into discrete words, punctuation, etc.

This breaks the text down into discete units, separating punctuation from words.

We can also use nltk to do tokenization for us:

Use redex for this.

In [19]:
import nltk
# "natural language took kit"
# this handles all sorts of word and language stuffs.
# it may require a list of words

Make the thing:

In [20]:
tokenizer = nltk.tokenize.ToktokTokenizer()

Use the thing

In [21]:
print(tokenizer.tokenize(original, return_str=True))

The rumors are true ! The time has arrived. Codeup has officially opened applications to our new Data Science career accelerator , with only 25 seats available ! This immersive program is one of a kind in San Antonio , and will help you land a job in Glassdoor ’ s #1 Best Job in America.
Data Science is a method of providing actionable intelligence from data. The data revolution has hit San Antonio , resulting in an explosion in Data Scientist positions across companies like USAA , Accenture , Booz Allen Hamilton , and HEB. We ’ ve even seen UTSA invest $ 70 M for a Cybersecurity Center and School of Data Science. We built a program to specifically meet the growing demands of this industry.
Our program will be 18 weeks long , full-time , hands-on , and project-based. Our curriculum development and instruction is led by Senior Data Scientist , Maggie Giust , who has worked at HEB , Capital Group , and Rackspace , along with input from dozens of practitioners and hiring partners. Student

### Stemming and Lemmatization
Usually you will want to use lemmatization, but we'll look at both.

Below here is Stemming

In [22]:
# Create the nltk stemmer object, then use it
ps = nltk.stem.PorterStemmer()

In [25]:
ps.stem('calls'), ps.stem('called'), ps.stem('calling')

('call', 'call', 'call')

In [23]:
[ps.stem(word) for word in ['call','called','calling']]

['call', 'call', 'call']

Now we can apply this stemming transformation to all the words in the article.

In [36]:
stems = [ps.stem(word) for word in article.split()]
article_stemmed = ' '.join(stems)

In [37]:
print(article_stemmed)

the rumor are true the time ha arriv codeup ha offici open applic to our new data scienc career acceler with onli 25 seat avail thi immers program is one of a kind in san antonio and will help you land a job in glassdoor 1 best job in america data scienc is a method of provid action intellig from data the data revolut ha hit san antonio result in an explos in data scientist posit across compani like usaa accentur booz allen hamilton and heb weve even seen utsa invest 70 m for a cybersecur center and school of data scienc we built a program to specif meet the grow demand of thi industri our program will be 18 week long fulltim handson and projectbas our curriculum develop and instruct is led by senior data scientist maggi giust who ha work at heb capit group and rackspac along with input from dozen of practition and hire partner student will work with real data set realist problem and the entir data scienc pipelin from collect to deploy they will receiv profession develop train in resum

Above is how to do this in steps.  Below is a list comprehension to make it more streamlined. 

In [38]:
#print(' '.join([ps.stem(word) for word in article.split()]))

In [39]:
pd.Series(stems).value_counts()[:10]

data      13
and       13
to         9
a          8
in         8
scienc     7
our        7
learn      6
the        6
with       6
dtype: int64

#### Now, below here is Lemmatization, which we'll be using.

Lemmatization is very similar to stemming, however, the base form in this case is known as the root word, but not the root stem. The difference is that the root word is always a lexicographically correct word (present in the dictionary), but the root stem may not be so. Thus, root word, also known as the lemma, will always be present in the dictionary.

Note that the lemmatization process is considerably slower than stemming, because an additional step is involved where the root form or lemma is formed by removing the affix from the word if and only if the lemma is present in the dictionary.

In [40]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rachelreuter/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [44]:
wnl = nltk.stem.WordNetLemmatizer()

In [45]:
for word in 'study studies'.split():
    print('stem:', ps.stem(word), '-- lemma:', wnl.lemmatize(word))

stem: studi -- lemma: study
stem: studi -- lemma: study


Now apply lemmatization to the entire document.

In [46]:
lemmas = [wnl.lemmatize(word.lower()) for word in article.split()]
article_lemmatized = ' '.join(lemmas)

In [47]:
print(article_lemmatized)

the rumor are true the time ha arrived codeup ha officially opened application to our new data science career accelerator with only 25 seat available this immersive program is one of a kind in san antonio and will help you land a job in glassdoors 1 best job in america data science is a method of providing actionable intelligence from data the data revolution ha hit san antonio resulting in an explosion in data scientist position across company like usaa accenture booz allen hamilton and heb weve even seen utsa invest 70 m for a cybersecurity center and school of data science we built a program to specifically meet the growing demand of this industry our program will be 18 week long fulltime handson and projectbased our curriculum development and instruction is led by senior data scientist maggie giust who ha worked at heb capital group and rackspace along with input from dozen of practitioner and hiring partner student will work with real data set realistic problem and the entire data

Now that we have a list of the lemmas, or actual root words, we can look at the most frequent words.

It generates the 'bag of words' we spoke of earlier, with a list of words and their respective counts.

In [48]:
pd.Series(lemmas).value_counts()[:10]

data       13
and        13
to          9
in          8
a           8
science     7
our         7
will        6
of          6
the         6
dtype: int64

### Removing Stopwords

Words which have little or no significance, especially when constructing meaningful features from text, are known as stop words (or stopwords). These are usually words that end up having the maximum frequency if you do a simple term or word frequency in a corpus. Typically, these can be articles, conjunctions, prepositions and so on. Some examples of stopwords are a, an, the, and like.

While there is no universal stopword list, we will use a standard English language stopwords list from nltk. You can also add your own domain-specific stopwords as needed.

Before removing stopwords, we want to segment text into linguistic units such as words or numbers. This process is called tokenization.

#### Tokenization

In [49]:
stopword_list = stopwords.words('english')

In [50]:
stopword_list.remove('no')
stopword_list.remove('not')

In [51]:
stopword_list[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [53]:
# without_stopwords = [word for word in lemmas if word not in stopword_list]

In [54]:
words = article.split()
filtered_words = [w for w in words if w not in stopword_list]

In [55]:
filtered_words

['rumors',
 'true',
 'time',
 'arrived',
 'codeup',
 'officially',
 'opened',
 'applications',
 'new',
 'data',
 'science',
 'career',
 'accelerator',
 '25',
 'seats',
 'available',
 'immersive',
 'program',
 'one',
 'kind',
 'san',
 'antonio',
 'help',
 'land',
 'job',
 'glassdoors',
 '1',
 'best',
 'job',
 'america',
 'data',
 'science',
 'method',
 'providing',
 'actionable',
 'intelligence',
 'data',
 'data',
 'revolution',
 'hit',
 'san',
 'antonio',
 'resulting',
 'explosion',
 'data',
 'scientist',
 'positions',
 'across',
 'companies',
 'like',
 'usaa',
 'accenture',
 'booz',
 'allen',
 'hamilton',
 'heb',
 'weve',
 'even',
 'seen',
 'utsa',
 'invest',
 '70',
 'cybersecurity',
 'center',
 'school',
 'data',
 'science',
 'built',
 'program',
 'specifically',
 'meet',
 'growing',
 'demands',
 'industry',
 'program',
 '18',
 'weeks',
 'long',
 'fulltime',
 'handson',
 'projectbased',
 'curriculum',
 'development',
 'instruction',
 'led',
 'senior',
 'data',
 'scientist',
 'maggie'

In [56]:
print('Removed {} stopwords'.format(len(words) - len(filtered_words)))
print('---')

Removed 122 stopwords
---


In [57]:
article_without_stopwords = ' '.join(filtered_words)

In [58]:
print(article_without_stopwords)

rumors true time arrived codeup officially opened applications new data science career accelerator 25 seats available immersive program one kind san antonio help land job glassdoors 1 best job america data science method providing actionable intelligence data data revolution hit san antonio resulting explosion data scientist positions across companies like usaa accenture booz allen hamilton heb weve even seen utsa invest 70 cybersecurity center school data science built program specifically meet growing demands industry program 18 weeks long fulltime handson projectbased curriculum development instruction led senior data scientist maggie giust worked heb capital group rackspace along input dozens practitioners hiring partners students work real data sets realistic problems entire data science pipeline collection deployment receive professional development training resume writing interviewing continuing education prepare smooth transition workforce focus applied data science immediate i